In [ ]:
# %% [markdown]
# ## 1. Instalación de dependencias
# **MODIFICADO:** Se agregó Tesseract OCR y se removió Anthropic

# %%
# Instalación de Tesseract OCR y bibliotecas necesarias
!apt-get install tesseract-ocr tesseract-ocr-spa -y -qq
!pip install pytesseract pandas matplotlib seaborn pillow opencv-python -q

print("✓ Instalación completada")


✓ Instalación completada


In [ ]:
# %% [markdown]
# ## 2. Importación de bibliotecas

# %%
import anthropic
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import json
import base64
import io
import os
from datetime import datetime

# Configurar estilo de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")# %% [markdown]
# ## 2. Importación de bibliotecas

# %%
import anthropic
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import json
import base64
import io
import os
from datetime import datetime

# Configurar estilo de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

ModuleNotFoundError: No module named 'anthropic'

In [ ]:
# %% [markdown]
# ## 3. Configuración de la API de Claude

# %%
# IMPORTANTE: Configura tu API key de Anthropic
# Opción 1: Variable de entorno (recomendado)
# Opción 2: Directamente aquí (NO subir a GitHub con la key)

# Para Colab, puedes usar:
from google.colab import userdata
try:
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
except:
    # Si no está en Colab secrets, pídela al usuario
    ANTHROPIC_API_KEY = input("Ingresa tu API Key de Anthropic: ")

# Inicializar cliente
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

KeyboardInterrupt: Interrupted by user

In [ ]:
# %% [markdown]
# ## 5. Carga y visualización de la imagen

# %%
# Ruta de la imagen del periódico
IMAGE_PATH = "data/el_martillo/page_01.png"

# Crear directorio si no existe
os.makedirs(os.path.dirname(IMAGE_PATH), exist_ok=True)

# NOTA: Debes descargar una página del periódico El Martillo de:
# https://fuenteshistoricasdelperu.com/2020/12/06/el-martillo-chiclayo-1903-1919/
# y guardarla en: data/el_martillo/page_01.png

# Verificar que existe la imagen
if os.path.exists(IMAGE_PATH):
    print(f"✓ Imagen encontrada en: {IMAGE_PATH}")
    img = display_image(IMAGE_PATH)
    print(f"Dimensiones: {img.size[0]}x{img.size[1]} pixels")
else:
    print(f"⚠ ADVERTENCIA: No se encontró la imagen en {IMAGE_PATH}")
    print("Por favor, descarga una página del periódico y guárdala en esa ubicación.")


In [ ]:
# %% [markdown]
# ## 6. Extracción de texto con Claude OCR

# %%
def extract_text_with_claude(image_path):
    """
    Usa Claude Vision API para extraer texto de la imagen del periódico
    """
    # Codificar imagen
    image_data = encode_image_to_base64(image_path)
    media_type = get_image_media_type(image_path)

    # Prompt para OCR estructurado
    prompt = """Analiza esta página del periódico histórico "El Martillo" de Chiclayo, Perú (1903-1919).

Extrae TODA la información visible y estructúrala de la siguiente manera:

1. METADATOS DE LA PÁGINA:
   - Fecha de publicación (si es visible)
   - Número de edición/ejemplar (si es visible)
   - Año

2. CONTENIDO ESTRUCTURADO:
   Para cada artículo, sección o elemento de la página, proporciona:
   - Título/Encabezado principal
   - Sección (si está identificada: noticias, editorial, anuncios, avisos, etc.)
   - Tipo de contenido (artículo, anuncio, aviso legal, editorial, nota social, etc.)
   - Texto completo transcrito (preserva la ortografía original, incluso si tiene errores)

3. ANUNCIOS Y AVISOS:
   Lista todos los anuncios comerciales o avisos que aparezcan

Por favor, sé exhaustivo y transcribe todo el texto legible, incluyendo:
- Artículos completos
- Anuncios publicitarios
- Avisos legales
- Notas sociales
- Cualquier otro texto visible

Presenta la información en formato JSON con esta estructura:
{
  "metadata": {
    "date": "fecha si está visible",
    "issue_number": "número si está visible",
    "year": "año"
  },
  "content": [
    {
      "headline": "título del artículo/sección",
      "section": "nombre de la sección",
      "type": "artículo/anuncio/aviso/editorial/etc",
      "text_excerpt": "texto completo transcrito"
    }
  ]
}"""

    # Llamada a la API
    message = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=4096,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": media_type,
                            "data": image_data,
                        },
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ],
            }
        ],
    )

    # Extraer respuesta
    response_text = message.content[0].text
    return response_text

# Ejecutar OCR
print("Procesando imagen con Claude OCR...")
print("Esto puede tomar 30-60 segundos...\n")

if os.path.exists(IMAGE_PATH):
    raw_ocr_result = extract_text_with_claude(IMAGE_PATH)
    print("✓ OCR completado\n")
    print("Resultado extraído:")
    print("=" * 80)
    print(raw_ocr_result)
    print("=" * 80)
else:
    print("⚠ No se puede ejecutar OCR sin la imagen")
    raw_ocr_result = None

In [ ]:
# %% [markdown]
# ## 7. Procesamiento y limpieza de datos

# %%
def parse_ocr_json(ocr_text):
    """
    Parsea el resultado del OCR y lo convierte a estructura de datos
    """
    try:
        # Intentar extraer JSON del texto
        # A veces Claude incluye el JSON entre ```json y ```
        if "```json" in ocr_text:
            start = ocr_text.find("```json") + 7
            end = ocr_text.find("```", start)
            json_text = ocr_text[start:end].strip()
        elif "```" in ocr_text:
            start = ocr_text.find("```") + 3
            end = ocr_text.find("```", start)
            json_text = ocr_text[start:end].strip()
        else:
            json_text = ocr_text

        # Parsear JSON
        data = json.loads(json_text)
        return data
    except json.JSONDecodeError as e:
        print(f"Error al parsear JSON: {e}")
        print("Intentando extraer datos manualmente...")
        return None

def create_dataframe_from_ocr(ocr_data):
    """
    Convierte los datos del OCR en un DataFrame de pandas
    """
    if ocr_data is None:
        return None

    # Extraer metadatos
    metadata = ocr_data.get('metadata', {})
    date = metadata.get('date', 'N/A')
    issue_number = metadata.get('issue_number', 'N/A')
    year = metadata.get('year', 'N/A')

    # Crear lista de registros
    records = []
    for item in ocr_data.get('content', []):
        record = {
            'date': date,
            'issue_number': issue_number,
            'headline': item.get('headline', ''),
            'section': item.get('section', 'N/A'),
            'type': item.get('type', 'N/A'),
            'text_excerpt': item.get('text_excerpt', '')
        }
        records.append(record)

    # Crear DataFrame
    df = pd.DataFrame(records)
    return df

# Procesar datos
if raw_ocr_result:
    print("Procesando datos extraídos...")
    parsed_data = parse_ocr_json(raw_ocr_result)

    if parsed_data:
        df = create_dataframe_from_ocr(parsed_data)
        print(f"\n✓ DataFrame creado con {len(df)} registros\n")
        print("Vista previa de los datos:")
        print(df.head())
        print(f"\nColumnas: {list(df.columns)}")
    else:
        print("⚠ No se pudo crear el DataFrame automáticamente")
        df = None
else:
    df = None

In [ ]:
# %% [markdown]
# ## 8. Limpieza y normalización de datos

# %%
def clean_and_normalize_data(df):
    """
    Limpia y normaliza los datos extraídos
    """
    if df is None or df.empty:
        return None

    df_clean = df.copy()

    # Limpiar espacios en blanco
    for col in df_clean.columns:
        if df_clean[col].dtype == 'object':
            df_clean[col] = df_clean[col].str.strip()

    # Normalizar tipos de contenido
    type_mapping = {
        'articulo': 'artículo',
        'noticia': 'artículo',
        'nota': 'artículo',
        'aviso': 'anuncio',
        'publicidad': 'anuncio',
        'editorial': 'editorial',
        'otro': 'otro'
    }

    df_clean['type'] = df_clean['type'].str.lower()
    df_clean['type'] = df_clean['type'].replace(type_mapping)

    # Limitar longitud de extractos para visualización
    df_clean['text_length'] = df_clean['text_excerpt'].str.len()

    # Agregar fecha de procesamiento
    df_clean['processing_date'] = datetime.now().strftime('%Y-%m-%d')

    return df_clean

if df is not None:
    df_clean = clean_and_normalize_data(df)
    print("✓ Datos limpiados y normalizados\n")
    print("Estadísticas básicas:")
    print(df_clean.describe(include='all'))
else:
    df_clean = None

In [ ]:
# %% [markdown]
# ## 9. Guardado de datos estructurados

# %%
# Guardar CSV
if df_clean is not None:
    CSV_PATH = "data/el_martillo/el_martillo_structured_data.csv"
    df_clean.to_csv(CSV_PATH, index=False, encoding='utf-8-sig')
    print(f"✓ Datos guardados en: {CSV_PATH}")

    # También guardar JSON
    JSON_PATH = "data/el_martillo/el_martillo_structured_data.json"
    df_clean.to_json(JSON_PATH, orient='records', force_ascii=False, indent=2)
    print(f"✓ Datos guardados en: {JSON_PATH}")
else:
    print("⚠ No hay datos para guardar")


In [ ]:
# %% [markdown]
# ## 10. Análisis exploratorio de datos

# %%
if df_clean is not None and not df_clean.empty:
    print("ANÁLISIS EXPLORATORIO DE DATOS")
    print("=" * 80)

    # Conteo por tipo de contenido
    print("\n1. Distribución por tipo de contenido:")
    type_counts = df_clean['type'].value_counts()
    print(type_counts)

    # Conteo por sección
    print("\n2. Distribución por sección:")
    section_counts = df_clean['section'].value_counts()
    print(section_counts)

    # Estadísticas de longitud de texto
    print("\n3. Estadísticas de longitud de texto:")
    print(df_clean['text_length'].describe())

    # Artículos más largos
    print("\n4. Top 5 artículos más extensos:")
    top_articles = df_clean.nlargest(5, 'text_length')[['headline', 'type', 'text_length']]
    print(top_articles)

In [ ]:
# %% [markdown]
# ## 11. Visualizaciones

# %%
def create_visualizations(df):
    """
    Crea visualizaciones del contenido extraído
    """
    if df is None or df.empty:
        print("No hay datos para visualizar")
        return

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Análisis del Periódico El Martillo', fontsize=16, fontweight='bold')

    # 1. Distribución por tipo de contenido (gráfico de barras)
    ax1 = axes[0, 0]
    type_counts = df['type'].value_counts()
    type_counts.plot(kind='bar', ax=ax1, color='steelblue', edgecolor='black')
    ax1.set_title('Distribución por Tipo de Contenido', fontweight='bold')
    ax1.set_xlabel('Tipo de Contenido')
    ax1.set_ylabel('Cantidad')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(axis='y', alpha=0.3)

    # Agregar valores en las barras
    for i, v in enumerate(type_counts):
        ax1.text(i, v + 0.1, str(v), ha='center', va='bottom', fontweight='bold')

    # 2. Distribución por tipo (gráfico circular)
    ax2 = axes[0, 1]
    type_counts.plot(kind='pie', ax=ax2, autopct='%1.1f%%', startangle=90)
    ax2.set_title('Proporción de Tipos de Contenido', fontweight='bold')
    ax2.set_ylabel('')

    # 3. Distribución por sección
    ax3 = axes[1, 0]
    section_counts = df['section'].value_counts().head(10)
    section_counts.plot(kind='barh', ax=ax3, color='coral', edgecolor='black')
    ax3.set_title('Top 10 Secciones Más Frecuentes', fontweight='bold')
    ax3.set_xlabel('Cantidad')
    ax3.set_ylabel('Sección')
    ax3.grid(axis='x', alpha=0.3)

    # 4. Distribución de longitud de texto
    ax4 = axes[1, 1]
    df['text_length'].hist(bins=20, ax=ax4, color='lightgreen', edgecolor='black')
    ax4.set_title('Distribución de Longitud de Textos', fontweight='bold')
    ax4.set_xlabel('Longitud (caracteres)')
    ax4.set_ylabel('Frecuencia')
    ax4.grid(axis='y', alpha=0.3)

    # Agregar línea de media
    mean_length = df['text_length'].mean()
    ax4.axvline(mean_length, color='red', linestyle='--', linewidth=2, label=f'Media: {mean_length:.0f}')
    ax4.legend()

    plt.tight_layout()

    # Guardar figura
    plt.savefig('data/el_martillo/analisis_visual.png', dpi=300, bbox_inches='tight')
    print("✓ Visualización guardada en: data/el_martillo/analisis_visual.png")
    plt.show()

# Crear visualizaciones
if df_clean is not None:
    create_visualizations(df_clean)

In [ ]:
# %% [markdown]
# ## 12. Análisis de palabras frecuentes

# %%
from collections import Counter
import re

def analyze_frequent_words(df, top_n=20):
    """
    Analiza las palabras más frecuentes en los textos
    """
    if df is None or df.empty:
        return

    # Combinar todos los textos
    all_text = ' '.join(df['text_excerpt'].fillna('').astype(str))

    # Limpiar y tokenizar
    words = re.findall(r'\b[a-záéíóúñüA-ZÁÉÍÓÚÑÜ]{3,}\b', all_text.lower())

    # Palabras comunes a excluir (stopwords básicas en español)
    stopwords = {'que', 'de', 'la', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las',
                 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como',
                 'más', 'pero', 'sus', 'le', 'ya', 'o', 'fue', 'este', 'ha', 'si',
                 'porque', 'esta', 'son', 'entre', 'está', 'cuando', 'muy', 'sin',
                 'sobre', 'ser', 'tiene', 'también', 'me', 'hasta', 'hay', 'donde',
                 'han', 'quien', 'están', 'estado', 'desde', 'todo', 'nos', 'durante',
                 'estados', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'fueron',
                 'ese', 'eso', 'había', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes',
                 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto'}

    # Filtrar stopwords
    words = [w for w in words if w not in stopwords]

    # Contar frecuencias
    word_freq = Counter(words)
    most_common = word_freq.most_common(top_n)

    # Visualizar
    if most_common:
        words_list, counts_list = zip(*most_common)

        plt.figure(figsize=(12, 6))
        plt.barh(range(len(words_list)), counts_list, color='teal', edgecolor='black')
        plt.yticks(range(len(words_list)), words_list)
        plt.xlabel('Frecuencia', fontweight='bold')
        plt.ylabel('Palabra', fontweight='bold')
        plt.title(f'Top {top_n} Palabras Más Frecuentes en El Martillo', fontweight='bold', fontsize=14)
        plt.gca().invert_yaxis()
        plt.grid(axis='x', alpha=0.3)
        plt.tight_layout()
        plt.savefig('data/el_martillo/palabras_frecuentes.png', dpi=300, bbox_inches='tight')
        print(f"✓ Análisis de palabras guardado en: data/el_martillo/palabras_frecuentes.png")
        plt.show()

        print("\nPalabras más frecuentes:")
        for word, count in most_common[:15]:
            print(f"  {word}: {count} veces")

if df_clean is not None:
    analyze_frequent_words(df_clean, top_n=25)

In [ ]:
# %% [markdown]
# ## 13. Resumen y conclusiones

# %%
def generate_summary(df):
    """
    Genera un resumen estadístico del análisis
    """
    if df is None or df.empty:
        print("No hay datos para resumir")
        return

    print("=" * 80)
    print("RESUMEN DEL ANÁLISIS")
    print("=" * 80)
    print(f"\n📊 Total de elementos extraídos: {len(df)}")
    print(f"📅 Fecha de la página: {df['date'].iloc[0] if len(df) > 0 else 'N/A'}")
    print(f"📰 Número de edición: {df['issue_number'].iloc[0] if len(df) > 0 else 'N/A'}")

    print("\n📈 Distribución de contenido:")
    for tipo, cantidad in df['type'].value_counts().items():
        porcentaje = (cantidad / len(df)) * 100
        print(f"  • {tipo.capitalize()}: {cantidad} ({porcentaje:.1f}%)")

    print("\n📝 Estadísticas de texto:")
    print(f"  • Longitud promedio: {df['text_length'].mean():.0f} caracteres")
    print(f"  • Longitud mínima: {df['text_length'].min():.0f} caracteres")
    print(f"  • Longitud máxima: {df['text_length'].max():.0f} caracteres")

    print("\n🏷️ Secciones identificadas:")
    for seccion, cantidad in df['section'].value_counts().head(5).items():
        print(f"  • {seccion}: {cantidad} elementos")

    print("\n" + "=" * 80)

if df_clean is not None:
    generate_summary(df_clean)

In [ ]:
# %% [markdown]
# ## 14. Exportación final

# %%
print("\n📦 ARCHIVOS GENERADOS:")
print("=" * 80)

files_generated = [
    ("Datos estructurados (CSV)", "data/el_martillo/el_martillo_structured_data.csv"),
    ("Datos estructurados (JSON)", "data/el_martillo/el_martillo_structured_data.json"),
    ("Visualización principal", "data/el_martillo/analisis_visual.png"),
    ("Análisis de palabras", "data/el_martillo/palabras_frecuentes.png"),
    ("Imagen original", IMAGE_PATH)
]

for name, path in files_generated:
    if os.path.exists(path):
        size = os.path.getsize(path)
        print(f"✓ {name:30s} | {path:50s} | {size:,} bytes")
    else:
        print(f"⚠ {name:30s} | {path:50s} | NO ENCONTRADO")

print("=" * 80)
print("\n✅ Análisis completado exitosamente!")
print("\nPróximos pasos:")
print("1. Revisa el archivo CSV generado")
print("2. Examina las visualizaciones")
print("3. Completa el informe en formato Markdown")
print("4. Sube todos los archivos a tu repositorio de GitHub")

# %%